# Project - Course 09 - Reinforcement Learning - Cohort 10
## Anderson Yoshizato

## Deep Q Learning Stock Trading

Prepare an agent by implementing Deep Q-Learning that can perform unsupervised trading in stock trade. The aim of this project is to train an agent that uses Q-learning and neural networks to predict the profit or loss by building a model and implementing it on a dataset that is available for evaluation.


The stock trading index environment provides the agent with a set of actions:<br>
* Buy<br>
* Sell<br>
* Sit

This project has following sections:
* Import libraries 
* Create a DQN agent
* Preprocess the data
* Train and build the model
* Evaluate the model and agent
<br><br>

**Steps to perform**<br>

In the section **create a DQN agent**, create a class called agent where:
* Action size is defined as 3
* Experience replay memory to deque is 1000
* Empty list for stocks that has already been bought
* The agent must possess the following hyperparameters:<br>
  * gamma= 0.95<br>
  * epsilon = 1.0<br>
  * epsilon_final = 0.01<br>
  * epsilon_decay = 0.995<br>


    Note: It is advised to compare the results using different values in hyperparameters.

* Neural network has 3 hidden layers
* Action and experience replay are defined




In [1]:
# Import Lbraries
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam
import numpy as np
import random
from collections import deque




### Create a DQN Agent

In [ ]:
# Action space include 3 actions: Buy, Sell, and Sit
#Setting up the experience replay memory to deque with 1000 elements inside it
#Empty list with inventory is created that contains the stocks that were already bought
#Setting up gamma to 0.95, that helps to maximize the current reward over the long-term
#Epsilon parameter determines whether to use a random action or to use the model for the action. 
#In the beginning random actions are encouraged, hence epsilon is set up to 1.0 when the model is not trained.
#And over time the epsilon is reduced to 0.01 in order to decrease the random actions and use the trained model
#We're then set the speed of decreasing epsililon in the epsilon_decay parameter

#Defining our neural network:
#Define the neural network function called _model and it just takes the keyword self
#Define the model with Sequential()
#Define states i.e. the previous n days and stock prices of the days
#Defining 3 hidden layers in this network
#Changing the activation function to relu because mean-squared error is used for the loss

# Creating Class
class Agent:
# Initializing Variables
    def __init__(self, state_size):
        self.state_size = state_size
        self.action_size = 3
        self.memory = deque(maxlen=1000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.01
        self.model = self._model()
        self.target_model = self._model()
        self.update_target_model()
        self.inventory = []
        
# Creating the model
    def _model(self):
        model = Sequential()
        model.add(Dense(32, input_dim = self.state_size, activation='relu'))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss="mse", optimizer=Adam(lr=self.learning_rate))
        return model
         
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def remember(self_action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size +1, l):
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
              
    def save(self, name):
            self.model.save_weights(name)


### Preprocess the stock market data

In [2]:
import math

# prints formatted price
def formatPrice(n):
	return ("-$" if n < 0 else "$") + "{0:.2f}".format(abs(n))

# returns the vector containing stock data from a fixed file
def getStockDataVec(key):
	vec = []
	lines = open("" + key + ".csv", "r").read().splitlines()

	for line in lines[1:]:
		vec.append(float(line.split(",")[4]))

	return vec

# returns the sigmoid
def sigmoid(x):
	return 1 / (1 + math.exp(-x))

# returns an an n-day state representation ending at time t
def getState(data, t, n):
	d = t - n + 1
	block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
	res = []
	for i in range(n - 1):
		res.append(sigmoid(block[i + 1] - block[i]))

	return np.array([res])

### Train and build the model

In [3]:
import sys

#if len(sys.argv) != 4:
#	print ("Usage: python train.py [stock] [window] [episodes]")
#	exit()

#"""stock_name = input("Enter stock_name, window_size, Episode_count")"""
#Fill the given information when prompted: 
#Enter stock_name = GSPC_Training_Dataset
#window_size = 10
#Episode_count = 100 or it can be 10 or 20 or 30 and so on.

#***Start Point
#"""window_size = input()
#episode_count = input()"""
stock_name = "GSPC_Training_Dataset"
window_size = int(10)
episode_count = int(4)

agent = Agent(window_size)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32

for e in range(episode_count + 1):
	print ("Episode " + str(e) + "/" + str(episode_count))
	state = getState(data, 0, window_size + 1)

	total_profit = 0
	agent.inventory = []

	for t in range(l):
		action = agent.act(state)

		# sit
		next_state = getState(data, t + 1, window_size + 1)
		reward = 0

		if action == 1: # buy
			agent.inventory.append(data[t])
			print ("Buy: " + formatPrice(data[t]))

		elif action == 2 and len(agent.inventory) > 0: # sell
			bought_price = agent.inventory.pop(0)
			reward = max(data[t] - bought_price, 0)
			total_profit += data[t] - bought_price
			print ("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

		done = True if t == l - 1 else False
		agent.memory.append((state, action, reward, next_state, done))
		state = next_state

		if done:
			print ("--------------------------------")
			print ("Total Profit: " + formatPrice(total_profit))
			

		if len(agent.memory) > batch_size:
			agent.expReplay(batch_size)

	#if e % 10 == 0:
		agent.model.save("model_ep" + str(e))
        

Episode 0/4
Buy: $1283.27
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_ep0\assets
Sell: $1347.56 | Profit: $64.29
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1295.86
INFO:tensorflow:Assets written to: model_ep0\assets
Sell: $1300.80 | Profit: $4.94
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: mode

INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1223.14
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1225.35
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1218.60
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1224.38
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1234.45
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1219.24
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
Sell: $1198.78 | Profit: -$68.65
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1181.52
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1180.18
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1208.14
INFO:tens

INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1144.65
INFO:tensorflow:Assets written to: model_ep0\assets
Sell: $1149.37 | Profit: -$32.15
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1157.13
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1161.02
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1148.08
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1154.67
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1165.27
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1172.51
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1164.89
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1160.71
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1155.14
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1156.55
INFO:tensorflow:Assets written to: model_ep0\assets
Sell: $1145.60 | Profit: -$34.58
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $1138.41
INFO:tensorflow:Assets written to: mod

INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $920.47
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $917.93
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $881.56
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $847.75
INFO:tensorflow:Assets written to: model_ep0\assets
Sell: $819.85 | Profit: -$370.31
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $898.96
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:As

Buy: $861.40
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $844.61
INFO:tensorflow:Assets written to: model_ep0\assets
Sell: $855.70 | Profit: -$162.91
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $843.59
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
Sell: $835.97 | Profit: -$202.58
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $829.20
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
INFO:tensorflow:Assets written to: model_ep0\assets
Buy: $851.17
INFO:tensorflow:Assets written to: mo

...

INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorf

INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorf

INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorf

INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorf

INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorflow:Assets written to: model_ep4\assets
INFO:tensorf

### Evaluate the model and agent

In [7]:
import sys
from keras.models import load_model

stock_name = "GSPC_Evaluation_Dataset"
model_name = "model_ep4"

model = load_model("" + model_name)
window_size = model.layers[0].input.shape.as_list()[1]

agent = Agent(window_size)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32

state = getState(data, 0, window_size + 1)
total_profit = 0
agent.inventory = []

for t in range(l):
	action = agent.act(state)

	# sit
	next_state = getState(data, t + 1, window_size + 1)
	reward = 0

	if action == 1: # buy
		agent.inventory.append(data[t])
		print ("Buy: " + formatPrice(data[t]))

	elif action == 2 and len(agent.inventory) > 0: # sell
		bought_price = agent.inventory.pop(0)
		reward = max(data[t] - bought_price, 0)
		total_profit += data[t] - bought_price
		print ("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

	done = True if t == l - 1 else False
	agent.memory.append((state, action, reward, next_state, done))
	state = next_state

	if done:
		print ("--------------------------------")
		print (stock_name + " Total Profit: " + formatPrice(total_profit))


Buy: $1276.56
Sell: $1273.85 | Profit: -$2.71
Buy: $1285.96
Sell: $1283.76 | Profit: -$2.20
Buy: $1295.02
Buy: $1280.26
Sell: $1283.35 | Profit: -$11.67
Buy: $1290.84
Buy: $1291.18
Sell: $1296.63 | Profit: $16.37
Buy: $1286.12
Sell: $1307.59 | Profit: $16.75
Sell: $1304.03 | Profit: $12.85
Buy: $1307.10
Buy: $1310.87
Buy: $1324.57
Sell: $1321.87 | Profit: $35.75
Sell: $1343.01 | Profit: $35.91
Sell: $1315.44 | Profit: $4.57
Buy: $1307.40
Buy: $1306.10
Buy: $1319.88
Sell: $1327.22 | Profit: $2.65
Buy: $1306.33
Buy: $1330.97
Sell: $1321.15 | Profit: $13.75
Sell: $1310.13 | Profit: $4.03
Sell: $1321.82 | Profit: $1.94
Buy: $1320.02
Sell: $1295.11 | Profit: -$11.22
Sell: $1304.28 | Profit: -$26.69
Buy: $1296.39
Buy: $1273.72
Buy: $1279.21
Sell: $1298.38 | Profit: -$21.64
Sell: $1313.80 | Profit: $17.41
Sell: $1310.19 | Profit: $36.47
Sell: $1328.26 | Profit: $49.05
Buy: $1328.17
Buy: $1314.16
Buy: $1314.52
Buy: $1312.62
Buy: $1337.38
Buy: $1347.24
Sell: $1355.66 | Profit: $27.49
Buy: $1360

### Note: Run the training section for considerable episodes so that while evaluating the model it can generate significant profit.

## Conclusion:


### Becuase of the time spent to run each episode, it was not possible to tune the model parameters properly to check eventual overfitting and improve the overall profit. 

### Also, the number of episodes was limited to 5, which has prevented to improve the agent learning.

### This might explain the low performance of the model in terms of Profit generation.

### Besides the model tuning and increment in the number of episodes, the addition of other variables in the environment may improve the model performance.